# Reservoir calibration with SCE-UA
***

**Autor:** Chus Casado<br>
**Date:** 20-06-2024<br>

**Introduction:**<br>
This code calibrates a reservoir model using the genetic algorithm SCE-UA (Shuffle Complex Evolution-University of Arizona) ([Duan et al., 2023](https://link.springer.com/article/10.1007/BF00939380)).

The setup of the calibration (reservoir model, target variable(s), etc) is defined in the _config*.yml_ file.

**To do:**<br>
* [ ] Make sure that all the input data is defined in the configuration file.
* [ ] Convert this notebook into a executable script with several arguments:
    * [ ] `--config-file` to define the configuration file
    * [ ] `--id` to define a list of reservoirs to be calibrated. By default it would include all the reservoirs in the input data.
    * [ ] `--overwrite` to allow for overwriting the results of previous calibrations. By default is False, so the calibration would skip a reservoir if already calibrated with the same setup.
* [ ] Should we apply split sampling or not? If yes, what would be the best parameter set, the one with the best performance in the calibration or the validation set?
    
All these arguments can be added to the configuration file, instead.

**Questions:**<br>

In [1]:
import sys
sys.path.append('../../src/')
import os
os.environ['USE_PYGEOS'] = '0'
import numpy as np
import pandas as pd
import geopandas as gpd
import seaborn as sns
from datetime import datetime, timedelta
import spotpy
# from spotpy.objectivefunctions import kge
import yaml
from pathlib import Path
from tqdm.auto import tqdm

from lisfloodreservoirs.calibration import get_calibrator
from lisfloodreservoirs.models import get_model
from lisfloodreservoirs.utils.metrics import KGEmod

## Configuration

In [15]:
with open('config_linear_2var.yml', 'r', encoding='utf8') as ymlfile:
    cfg = yaml.load(ymlfile, Loader=yaml.FullLoader)

### Paths
PATH_GLOFAS = Path(cfg['paths']['GloFAS']['root'])
# PATH_GLOFAS_TS = PATH_GLOFAS / cfg['paths']['GloFAS']['timeseries']
PATH_RESOPS = Path(cfg['paths']['ResOpsUS']['root'])
PATH_RESOPS_TS = PATH_RESOPS / cfg['paths']['ResOpsUS']['timeseries']
PATH_GRAND = Path(cfg['paths']['GRanD'])

### Reservoir model
MODEL = cfg['simulation']['model'].lower()
MODEL_CFG = cfg['simulation'].get('config', {})

# calibration
ALGORITHM = cfg['calibration']['algorithm'].lower()
TARGET = cfg['calibration']['target']
MAX_ITER = cfg['calibration'].get('max_iter', 1000)
COMPLEXES = cfg['calibration'].get('COMPLEXES', 4)
TRAIN_SIZE = cfg['calibration'].get('TRAIN_SIZE', 0.7)
# # sequential mode
# parallel = "seq"  

# results will be saved in this path
PATH_OUT = Path('./') / MODEL / 'calibration' / ALGORITHM
if len(TARGET) == 1:
    PATH_OUT = PATH_OUT / 'univariate' / TARGET[0]
elif len(TARGET) == 2:
    PATH_OUT /= 'bivariate'
else:
    print('ERROR. Only univariate or bivariate calibrations are supported')
    sys.exit()
PATH_OUT.mkdir(parents=True, exist_ok=True)
print(f'Results will be saved in {PATH_OUT}')

Results will be saved in linear\calibration\sceua\bivariate


## Data

### Reservoirs

#### GloFAS

In [6]:
# load shapefile of GloFAS reservoirs
glofas_res = gpd.read_file(PATH_GLOFAS / 'tables' / 'GloFAS_reservoirs.shp')
glofas_res.rename(columns={'stor': 'CAP_GLWD'}, inplace=True)
print(f'GloFASv4 contains {glofas_res.shape[0]} reservoirs worldwide')

# remove those without GRAND_ID
glofas_res = glofas_res.loc[~glofas_res.GRAND_ID.isnull()]
glofas_res.GRAND_ID = glofas_res.GRAND_ID.astype(int)
glofas_res.set_index('GRAND_ID', drop=False, inplace=True)
print(f'{glofas_res.shape[0]} of those reservoirs have a GRAND_ID assigned')

# # select only those reservoirs included in ResOpsUS
# resopsus = gpd.read_file('../../GIS/reservoirs_GloFAS_ResOpsUS.shp')
# resopsus.set_index('ResID', drop=True, inplace=True)
# mask = reservoirs.index.intersection(resopsus.index)
# reservoirs = reservoirs.loc[mask]
# reservoirs.GRAND_ID = reservoirs.GRAND_ID.astype(int)



GloFASv4 contains 685 reservoirs worldwide
655 of those reservoirs have a GRAND_ID assigned


In [7]:
# # load shapefile of GloFAS reservoirs
# reservoirs = gpd.read_file('../../GIS/reservoirs_analysis_US.shp')
# reservoirs.set_index('ResID', drop=True, inplace=True)

# print(f'{reservoirs.shape[0]} reservoirs in the shape file')

#### GRanD


In [8]:
# load GRanD data set
grand = gpd.read_file(PATH_GRAND / 'grand_dams_v1_3.shp')
grand.set_index('GRAND_ID', drop=True, inplace=True)
grand = grand.replace(-99, np.nan)
print(f'GraND contains {grand.shape[0]} reservoirs worldwide')

# filter reservoirs represented in GloFAS4
mask_glofas = grand.index.intersection(glofas_res.index)
grand = grand.loc[mask_glofas]
print(f'{len(mask_glofas)} reservoirs are both in GloFASv4 and GRanD worldwide')

# add GRanD capacity to the reservoirs
glofas_res.loc[mask_glofas, 'CAP_GRAND'] = grand.loc[mask_glofas, 'CAP_MCM'].values

GraND contains 7320 reservoirs worldwide
655 reservoirs are both in GloFASv4 and GRanD worldwide


#### ResOpsUS

In [9]:
resops_res = gpd.read_file(PATH_RESOPS / 'raw' / 'GIS' / 'reservoirs.shp')
resops_res.set_index('DAM_ID', drop=True, inplace=True)
resops_res.index.name = 'GRAND_ID'
print(f'ResOpsUS contains {resops_res.shape[0]} reservoirs')

# filter reservoirs represented in GloFAS4
mask_glofas = resops_res.index.intersection(glofas_res.index)
resops_res = resops_res.loc[mask_glofas]
print(f'{len(mask_glofas)} reservoirs are both in GloFASv4 and ResOpsUS')

# filter reservoirs with observed storage and outflow
mask_ts = resops_res.STORAGE & resops_res.OUTFLOW
resops_res = resops_res.loc[mask_ts == 1]
print(f'Of those, {mask_ts.sum()} reservoirs have records of both storage and outflow')

glofas_res = glofas_res.loc[resops_res.index]
# glofas_res.set_index('ResID', inplace=True, drop=True)

ResOpsUS contains 677 reservoirs
121 reservoirs are both in GloFASv4 and ResOpsUS
Of those, 102 reservoirs have records of both storage and outflow


### Time series

#### GloFASv4

In [12]:
path_ts_sim = PATH_RESOPS / 'ancillary' / 'LiSFLOOD'
glofas_ts = {}
for ID in tqdm(glofas_res.index): # ID refers to GRanD
    ResID = glofas_res.loc[ID, 'ResID']
    file = path_ts_sim / f'{ResID:03}.csv'
    if file.is_file():
        df = pd.read_csv(file, parse_dates=True, dayfirst=False, index_col='date')
        # convert storage time series into volume
        df.storage *= glofas_res.loc[ID, 'CAP_GLWD'] * 1e6
        glofas_ts[ID] = df.copy()
    else:
        print(f"{file} doesn't exist")

print(f'{len(glofas_ts)} reservoirs in the GloFAS time series')

# period of GloFAS simulation
start, end = glofas_ts[ID].first_valid_index(), glofas_ts[ID].last_valid_index()

  0%|          | 0/102 [00:00<?, ?it/s]

Z:\nahaUsers\casadje\datasets\reservoirs\ResOpsUS\ancillary\LiSFLOOD\296.csv doesn't exist
Z:\nahaUsers\casadje\datasets\reservoirs\ResOpsUS\ancillary\LiSFLOOD\197.csv doesn't exist
Z:\nahaUsers\casadje\datasets\reservoirs\ResOpsUS\ancillary\LiSFLOOD\323.csv doesn't exist
Z:\nahaUsers\casadje\datasets\reservoirs\ResOpsUS\ancillary\LiSFLOOD\068.csv doesn't exist
Z:\nahaUsers\casadje\datasets\reservoirs\ResOpsUS\ancillary\LiSFLOOD\185.csv doesn't exist
97 reservoirs in the GloFAS time series


#### ResOpsUS

In [16]:
resops_ts = {}
for ID in tqdm(glofas_res.index): # ID refers to GRanD
    # load timeseries
    file = PATH_RESOPS_TS / f'ResOpsUS_{ID}.csv'
    if file.is_file():
        series_id = pd.read_csv(file, parse_dates=True, index_col='date')
    else:
        print(f"{file} doesn't exist")
    # remove empty time series
    series_id = series_id.loc[start:end]#.dropna(axis=1, how='all')
    # remove duplicated index
    series_id = series_id[~series_id.index.duplicated(keep='first')]
    # convert storage from hm3 to m3
    series_id.storage *= 1e6
    # save in dictionary
    resops_ts[ID] = series_id

print(f'{len(resops_ts)} reservoirs in the ResOpsUS time series')
    
# approximate the ResOpsUS reservoir capacity as the maximum value in the records
glofas_res['CAP_RESOPS'] = pd.Series({ID: df.storage.max() for ID, df in resops_ts.items()})

  0%|          | 0/102 [00:00<?, ?it/s]

102 reservoirs in the ResOpsUS time series


### Correct reservoir capacity

In [17]:
try:
    # import DataFrame with the fraction fill and the selected data source
    ff = pd.read_excel('fraction_fill.xlsx', index_col='ResID')
except:
    # create DataFrame with the fraction fill according to each data source
    ff = pd.DataFrame(columns=['GLOFAS', 'GRAND'], dtype=float)
    ff.index.name = 'ResID'
    for ID in glofas_res.index:
        ResID = cap_resops, cap_glofas, cap_grand = glofas_res.loc[ID, ['ResID', 'CAP_RESOPS', 'CAP_GLWD', 'CAP_GRAND']]
        if np.isnan(cap_resops):
            continue
        ff.loc[ResID, :] = cap_resops / cap_glofas, cap_resops / cap_grand
    # export
    ff.to_excel('fraction_fill.xlsx', index=True)

# define de capacity  ('CAP') as that of the most reliable source
glofas_res['CAP'] = np.nan
for ID in glofas_res.index:
    ResID = glofas_res.loc[ID, 'ResID']
    if ff.loc[ResID, 'selection'] == 'GLOFAS':
        glofas_res.loc[ID, 'CAP'] = glofas_res.loc[ID, 'CAP_GLWD']
    elif ff.loc[ResID, 'selection']:
        glofas_res.loc[ID, 'CAP'] = glofas_res.loc[ID, 'CAP_GRAND']

# convert storage time series into volume
for ID, df in glofas_ts.items():
    df.storage *= glofas_res.loc[ID, 'CAP'] * 1e6

## Calibration

In [ ]:
for ID in tqdm([393]):#reservoirs.index):
    
    # file where the calibration results will be saved
    dbname = f'{PATH_OUT}/{ID:03}_samples'
    if os.path.isfile(dbname + '.csv'):
        print(f'The file {dbname}.csv already exists.')
        continue   

    ## TIME SERIES
    try:
        # observed time series
        obs = resops_ts[ID][['storage', 'inflow', 'outflow']].copy()
        obs[obs < 0] = np.nan

        # define calibration period
        if obs.outflow.isnull().all():
            print(f'Reservoir {ID} is missing outflow records')
            continue
        elif obs.storage.isnull().all():
            print(f'Reservoir {ID} is missing storage records')
            continue
        else:
            start_obs = max([obs[var].first_valid_index() for var in ['storage', 'outflow']])
            end_obs = min([obs[var].last_valid_index() for var in ['storage', 'outflow']])
            cal_days = timedelta(days=np.floor((end_obs - start_obs).days * TRAIN_SIZE))
            start_cal = end_obs - cal_days

        # define train and test time series
        x_train = glofas_ts[ID].inflow[start_cal:end_obs]
        y_train = obs.loc[start_cal:end_obs, ['storage', 'outflow']]
        x_test = glofas_ts[ID].inflow[start:start_cal]
        y_test = obs.loc[start_obs:start_cal, ['storage', 'outflow']]
        
    except Exception as e:
        print(f'ERROR. The time series of reservoir {ID} could not be set up\n', e)
        continue

    ## SET UP SPOTPY
    try:
        # extract GloFAS reservoir parameters
        Vmin, Vtot, Qmin = glofas_res.loc[ID, ['clim', 'CAP', 'minq']]
        Vtot *= 1e6
        Vmin *= Vtot

        # initialize the calibration setup of the LISFLOOD reservoir routine
        setup = get_calibrator(MODEL,
                               inflow=x_train,
                               storage=y_train.storage, 
                               outflow=y_train.outflow,
                               Vmin=Vmin,
                               Vtot=Vtot,
                               Qmin=Qmin,
                               target=TARGET,
                               obj_func=KGEmod)

        # define the sampling method
        sceua = spotpy.algorithms.sceua(setup, dbname=dbname, dbformat='csv', save_sim=False)
    except Exception as e:
        print(f'ERROR. The SpotPY set up of reservoir {ID} could not be done\n', e)
        continue
        
    ## LAUNCH SAMPLING
    try:
        # start the sampler
        sceua.sample(MAX_ITER, ngs=COMPLEXES, kstop=3, pcento=0.01, peps=0.1)
    except Exception as e:
        print(f'ERROR. While sampling the reservoir {ID}\n', e)
        continue

    ### VALIDATION
    # read CSV of results
    try:
        results = pd.read_csv(f'{dbname}.csv')
        results.index.name = 'iteration'
        parcols = [col for col in results.columns if col.startswith('par')]
    except Exception as e:
        print(f'ERROR while reading results form reservoir {ID}\n', e)
        continue
    
    # compute validation KGE of each simulation and overwrite CSV file
    try:       
        results['like_val'] = np.nan
        for i in tqdm(results.index):
            sim = setup.simulation(pars=results.loc[i, parcols],
                                   inflow=x_test,
                                   storage_init=y_test.storage[0])
            results.loc[i, 'like_val'] = np.sqrt(np.sum([(1 - KGEmod(y_test[var], sim[var])[0])**2 for var in TARGET]))
        results.to_csv(f'{dbname}.csv', index=False, float_format='%.8f')
    except Exception as e:
        print(f'ERROR while computing KGE for the validation period in reservoir {ID}\n', e)
    
    # select optimal parameters (best validation) and export them
    try:
        best_iter = results.like_val.idxmin() # results.like1.idxmin()
        parvalues = {col[3:]: float(results.loc[best_iter, col]) for col in parcols}
        with open(f'{PATH_OUT}/{ID:03}_optimal_parameters.yml', 'w') as file:
            yaml.dump(parvalues, file)
    except Exception as e:
        print(f'ERROR while searching for optimal parameters in reservoir {ID}\n', e)
        continue
    
    # simulate the whole observed period with the optimal parameterization
    try:       
        if MODEL.lower() == 'linear':
            kwargs = {'Vmin': Vmin, 'Vtot': Vtot, 'Qmin': Qmin, 'T': parvalues['T']}
        elif MODEL.lower() == 'lisflood':
            Vf = parvalues['FFf'] * Vtot
            Vn = Vmin + parvalues['alpha'] * (Vf - Vmin)
            Vn_adj = Vn + parvalues['beta'] * (Vf - Vn)
            Qf = setup.inflow.quantile(parvalues['QQf'])
            Qn = parvalues['gamma'] * Qf
            k = parvalues['k']
            kwargs = {'Vmin': Vmin, 'Vn': Vn, 'Vn_adj': Vn_adj, 'Vf': Vf, 'Vtot': Vtot, 'Qmin': Qmin, 'Qn': Qn, 'Qf': Qf}
        else:
            raise ValueError(f'Model {MODEL} is not supported')
        res = get_model(MODEL, **kwargs)  
        sim = res.simulate(glofas_ts[ID].inflow[start_obs:end_obs],
                           obs.storage[start_obs])
        
        # performance
        performance = pd.DataFrame(index=['KGE', 'alpha', 'beta', 'rho'], columns=obs.columns)
        for var in performance.columns:
            try:
                performance[var] = KGEmod(obs[var], sim[var])
            except:
                continue
        file_out = PATH_OUT / f'{ID:03}_performance.csv'
        performance.to_csv(file_out, float_format='%.3f')
        
        res.scatter(sim,
                    obs,
                    norm=False,
                    title=ID,
                    save=PATH_OUT / f'{ID:03}_scatter.jpg'
                   )
        
        res.lineplot({'GloFAS': glofas_ts[ID], 'cal': sim},
                     obs,
                     save=PATH_OUT / f'{ID:03}_lineplot.jpg'
                    )
    except Exception as e:
        print(f'ERROR while simulating with optimal parameters in reservoir {ID}\n', e)

  0%|          | 0/1 [00:00<?, ?it/s]

Initializing the  Shuffled Complex Evolution (SCE-UA) algorithm  with  1000  repetitions
The objective function will be minimized
Starting burn-in sampling...


  0%|          | 0/9714 [00:00<?, ?it/s]

Initialize database...
['csv', 'hdf5', 'ram', 'sql', 'custom', 'noData']
* Database file 'linear\calibration\sceua\bivariate/393_samples.csv' created.


  0%|          | 0/9714 [00:00<?, ?it/s]

...

  0%|          | 0/9714 [00:00<?, ?it/s]

  0%|          | 0/9714 [00:00<?, ?it/s]

  0%|          | 0/9714 [00:00<?, ?it/s]

  0%|          | 0/9714 [00:00<?, ?it/s]

  0%|          | 0/9714 [00:00<?, ?it/s]